# Fine Tuning Roberta for Sentiment Analysis





### Introduction

In this tutorial I will be fine tuning a roberta model for the **Sentiment Analysis** problem with or without class_weights.

 - Hardware Requirements:
	 - Python 3.6 and above
	 - Pytorch, Transformers and All the stock Python ML Libraries
	 - GPU enabled setup

In [ ]:
!pip install transformers==3.0.2

In [1]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging, re, string
from sklearn import metrics
logging.basicConfig(level=logging.ERROR)

In [2]:
# Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [2]:
df = pd.read_csv("/kaggle/input/senti-dataset-7/tagged_selections_by_sentence.csv")


In [4]:
#cleaning the data by removing punctuation and redundant links
df['MergedSelections'] = df['MergedSelections'].apply(lambda x: x.lower())
df['MergedSelections'] = df['MergedSelections'].apply(lambda x: re.sub(r'http\S+|www.\S+', '', x))
df['MergedSelections'] = df['MergedSelections'].apply(lambda x: re.sub(r"\d+", "", x))
df['MergedSelections'] = df['MergedSelections'].apply(lambda x: re.sub(r'[{}()\[\]\-<>,;:.\'"\\"+=_\\\/?|]+|“|”|’'.format(string.punctuation),'', x))
df['MergedSelections'] = df['MergedSelections'].apply(lambda x: re.sub(r"[-+*/()^]\d+(?:\.\d+)?(?:[eE][-+]?\d+)?", "", x))

In [4]:
df['list'] = df[df.columns[7:]].values.tolist()
new_df = df[['MergedSelections', 'list']].copy()
new_df.head()
new_df = df[['MergedSelections', 'list']]

In [5]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 500
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.MergedSelections
        self.targets = self.data.list
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [7]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (6757, 2)
TRAIN Dataset: (5406, 2)
TEST Dataset: (1351, 2)


In [8]:
#Finding class_weights for the unbalanced dataset
test_data
print("\nTraining Data")
X_test = train_data["MergedSelections"]
y_test = train_data["list"]
class_weights = []

for k in range(7):
    n=[1 for i in range(len(X_test)) if y_test[i][k] ==1]
    print("{} data:".format(k), len(n))
    class_weights.append(train_data["MergedSelections"].shape[0]/(7*len(n)))
    
class_weights = class_weights
    
print("\nTesting Data")
X_test = test_data["MergedSelections"]
y_test = test_data["list"]
for k in range(7):
    tagged_data=[[X_test[i], y_test[i]] for i in range(len(X_test)) if y_test[i][k] ==1]
    print("{} data:".format(k), len(tagged_data))
    
print(class_weights)


Training Data
0 data: 636
1 data: 1686
2 data: 58
3 data: 234
4 data: 416
5 data: 1133
6 data: 92

Testing Data
0 data: 166
1 data: 398
2 data: 14
3 data: 59
4 data: 101
5 data: 310
6 data: 27
[1.2142857142857142, 0.45805795627859686, 13.31527093596059, 3.3003663003663, 1.856456043956044, 0.6816290505610894, 8.394409937888199]


In [9]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 10
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 10
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


<a id='section04'></a>
### Creating the Neural Network for Fine Tuning

#### Neural Network
 - We will be creating a neural network with the `RobertaClass`.
 - This network will have the Roberta Language model followed by a `dropout` and finally a `Linear` layer to obtain the final outputs.
 - The data will be fed to the Roberta Language model as defined in the dataset.
 - Final layer outputs is what will be compared to the `Sentiment category` to determine the accuracy of models prediction.
 - We will initiate an instance of the network called `model`. This instance will be used for training and then to save the final trained model for future inference.

#### Loss Function and Optimizer
 - `Loss Function` and `Optimizer` and defined in the next cell.
 - The `Loss Function` is used the calculate the difference in the output created by the model and the actual output.
 - `Optimizer` is used to update the weights of the neural network to improve its performance.

In [10]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 7)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [11]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

<a id='section05'></a>
### Fine Tuning the Model

After all the effort of loading and preparing the data and datasets, creating the model and defining its loss and optimizer. This is probably the easier steps in the process.

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network.

Following events happen in this function to fine tune the neural network:
- The dataloader passes data to the model based on the batch size.
- Subsequent output from the model and the actual category are compared to calculate the loss.
- Loss value is used to optimize the weights of the neurons in the network.
- After every 5000 steps the loss value is printed in the console.

As you can see just in 1 epoch by the final step the model was working with a loss of 0.8141926634122427.

In [12]:
# Creating the loss function and optimizer
# loss_function = torch.nn.CrossEntropyLoss()
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss(weight=torch.tensor(class_weights).to(device, dtype = torch.float))(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [13]:
import tqdm
def train(epoch):
    model.train()
    fin_targets=[]
    fin_outputs=[]
    for itr,data in tqdm.tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        fin_targets.extend(targets.cpu().detach().numpy().tolist())
        outputs_ = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
        outputs_ = np.array(outputs_) >= 0.5
        fin_outputs.extend(outputs_)
        if itr%100==0:
            accuracy = metrics.accuracy_score(fin_targets, fin_outputs)
            f1_score_micro = metrics.f1_score(fin_targets, fin_outputs, average='micro')
            f1_score_macro = metrics.f1_score(fin_targets, fin_outputs, average='macro')
            print(f'Epoch: {epoch}: {itr}, Loss:  {loss.item()}')
            print(f"Accuracy Score = {accuracy}, F1 Score (Micro) = {f1_score_micro}, F1 Score (Macro) = {f1_score_macro}")

        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()

def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [14]:
from sklearn.metrics import f1_score
EPOCHS = 20
for epoch in range(EPOCHS):
    train(epoch)
    outputs, targets = validation(epoch)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    # Calculate F1-score for each class (assuming multi-class classification)
    f1_scores = f1_score(targets, outputs, average=None)  # Get scores per class
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")
    for score in f1_scores:
        print(f"F1-score {score:.4f}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `

Epoch: 0: 0, Loss:  2.8606388568878174
Accuracy Score = 0.0, F1 Score (Micro) = 0.18181818181818182, F1 Score (Macro) = 0.17460317460317462


101it [01:09,  1.38it/s]

Epoch: 0: 100, Loss:  1.227628469467163
Accuracy Score = 0.30074257425742573, F1 Score (Micro) = 0.12007504690431521, F1 Score (Macro) = 0.07460291656998892


201it [02:22,  1.38it/s]

...

Epoch: 19: 500, Loss:  0.05314391106367111
Accuracy Score = 0.7836826347305389, F1 Score (Micro) = 0.8409459021703919, F1 Score (Macro) = 0.9156981694156245


601it [07:15,  1.34it/s]

Epoch: 19: 600, Loss:  0.0602487251162529
Accuracy Score = 0.78369384359401, F1 Score (Micro) = 0.8398150666304052, F1 Score (Macro) = 0.9166190709231834


676it [08:09,  1.38it/s]
/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explici

Accuracy Score = 0.5891931902294597
F1 Score (Micro) = 0.6458923512747876
F1 Score (Macro) = 0.5305263541838287
F1-score 0.9524
F1-score 0.6575
F1-score 0.0000
F1-score 0.4564
F1-score 0.9163
F1-score 0.4130
F1-score 0.3182


<a id='section06'></a>
### Validating the Model

During the validation stage we pass the unseen data(Testing Dataset) to the model. This step determines how good the model performs on the unseen data.

This unseen data is the 20% of `train.tsv` which was seperated during the Dataset creation stage.
During the validation stage the weights of the model are not updated. Only the final output is compared to the actual value. This comparison is then used to calcuate the accuracy of the model.

As you can see the model is predicting the correct category of a given sample to a 69.47% accuracy which can further be improved by training more.

<a id='section07'></a>
### Saving the Trained Model Artifacts for inference

This is the final step in the process of fine tuning the model.

The model and its vocabulary are saved locally. These files are then used in the future to make inference on new inputs of news headlines.

In [15]:
output_model_file = 'pytorch_roberta_sentiment_class_weight.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)
# model = load_model

('./vocab.json', './merges.txt')

In [ ]:
from sklearn.metrics import f1_score, multilabel_confusion_matrix
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")
cm = multilabel_confusion_matrix(targets, outputs)
n_samples = np.sum(cm, axis=0)
# Calculate individual class accuracy
class_accuracies = np.diagonal(cm) / n_samples  # Divide by class sample count


# Calculate F1-score for each class (assuming multi-class classification)
f1_scores = f1_score(targets, outputs, average=None)  # Get scores per class

# Print F1-score for each class with corresponding class label (assuming unique labels)
class_labels = [i for i in range(7)]  # Get unique class labels
print(class_accuracies)
for label, score in zip(class_labels, f1_scores):
    print(f"F1-score (Class {label}): {score:.4f}")
  # print(f"  Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score
y_true = np.array(targets)
y_pred = outputs
# Assuming you have true labels (y_true) and predicted labels (y_pred)

# Per-class accuracy
# Per-class accuracy
class_accuracies = []
for class_ in range(7):  # Iterate through all 7 classes
    class_predictions = y_pred[:, class_]  # Get predictions for this class
    class_labels = y_true[:, class_]     # Get ground truth labels for this class
    correct_predictions = np.sum(class_predictions == class_labels)
    class_accuracy = correct_predictions / len(class_labels)
    class_accuracies.append(class_accuracy)

# Overall accuracy (might not be ideal for multi-label problems)
accuracy = accuracy_score(y_true, y_pred)
print(class_accuracies)
print(accuracy)

In [16]:
import torch

# Load the model from the .bin file
model = torch.load("/kaggle/input/roberta-7/pytorch_roberta_sentiment.bin", map_location=torch.device('cuda'))


In [22]:
def infer(text):
    text = " ".join(text.split())

    inputs = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens = True,
        max_length = MAX_LEN,
        pad_to_max_length = True,
        return_token_type_ids = True
    )
    ids = [inputs['input_ids']]
    mask = [inputs['attention_mask']]
    token_type_ids = [inputs["token_type_ids"]]
    output = model(torch.tensor(ids, dtype=torch.long).to(device), torch.tensor(mask, dtype=torch.long).to(device), torch.tensor(token_type_ids, dtype=torch.long).to(device))
    output = torch.sigmoid(output).cpu().detach().numpy()
    output = np.array(output) >= 0.5
    return output

In [5]:
text = ""
tags = ["Greeting", "Backstory", "Justification", "Rant", "Gratitude", "Other", "Express Emotion"]
ans = infer(text)[0]
print(ans)
ans = [1 for i, val in enumerate(ans) if val==1]